In [1]:
import ee
import folium
# opens browser to request authenticatication token (OAUTH2) and store it.
ee.Authenticate()

# uses your stored auth token to login to ee.
ee.Initialize()

Enter verification code: 4/1AY0e-g7BzopIUtrH88qrawpeRSE9Hbu5eyPfJg14vqqC3zUNebb8PNEzjUk

Successfully saved authorization token.


In [2]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self

In [136]:
# load the raster Image dataset from ee
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# create map
hansen_map = EngineMap()

# add raster to map
hansen_map.add_ee_raster(gfc2014)

hansen_map

In [138]:
# tree cover
hansen_map2 = EngineMap()
hansen_map2.add_ee_raster(gfc2014, vis_params={"bands": ['treecover2000']})
hansen_map2

In [141]:
# tree cover, loss, gain
hansen_map3 = EngineMap()
hansen_map3.add_ee_raster(gfc2014, vis_params={"bands": ['loss', 'treecover2000', 'gain'],
                                              "max": [1, 255, 1],
                                              })
hansen_map3

In [144]:
# adding a mask 
hansen_map4 = EngineMap()
hansen_map4.add_ee_raster(gfc2014.mask(gfc2014), vis_params={"bands": ['treecover2000'],
                                                            "palette": ['000000', '00FF00'],
                                                            "max" :100})
hansen_map4

In [150]:
hansen_map5 = EngineMap()

# add tree cover raster
tree_cover = gfc2014.select("treecover2000")
hansen_map5.add_ee_raster(tree_cover.updateMask(tree_cover), vis_params={"palette" : ['000000', '00FF00'], 
                             "max" : 100})

# add loss raster
loss = gfc2014.select("loss")
hansen_map5.add_ee_raster(tree_cover.updateMask(loss), vis_params={"palette" : ['FF0000']})

#add gain raster
gain = gfc2014.select("gain")
hansen_map5.add_ee_raster(gain.updateMask(gain), vis_params={"palette" : ['0000FF']}, attr="Aparna")

hansen_map5
